In [1]:
import pandas as pd
import numpy as np
import csv
import keras
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D

Using TensorFlow backend.
C:\Users\zippy\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\zippy\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\zippy\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\zippy\anaconda3\envs\te

In [2]:
# This function should transform data into a usable format 
def featureProcess(path):
    data = pd.read_csv(path)
    #get audio data from vect1 to vect148
    #featureData = data.loc[:, "vect_1":"vect_148"]
    
    #get feature data from loudness to vect148
    featureData = data.loc[:, "loudness":"vect_148"]
    
    # normalization
    featureData=(featureData-featureData.mean())/featureData.std()
    
    return featureData.values.tolist()

In [3]:
# This function should transform data into a usable format 
def labelProcess(path):
    data = pd.read_csv(path)
    #get classes
    labels = data.loc[:,"genre"].values.tolist()
    
    
    return labels

In [4]:
def toOneHot(classes, labels):
    one_hot_labels = []

    for label in labels:
        one_hot_vector = [0] * len(classes)
        one_hot_vector[classes.index(label)] = 1

        one_hot_labels.append(one_hot_vector)
    return one_hot_labels

In [5]:
def LSTMModel(trainData, trainLabel):
    
    num_classes = len(trainLabel[0])  # total classes to train
    model = keras.models.Sequential()
    
    #all feature
    #model.add(Conv1D(128, 10,padding="same", input_shape=(148, 1), activation="relu", kernel_initializer="random_uniform"))
    
    #audio feature
    model.add(Conv1D(128, 10,padding="same", input_shape=(154, 1), activation="relu", kernel_initializer="random_uniform"))
    
    model.add(Dropout(0.25))
    model.add(MaxPooling1D(pool_size=(2)))
    model.add(Conv1D(64, 10,padding="same", activation="relu", kernel_initializer="random_uniform"))
    
    model.add(LSTM(50,  dropout=0.25, recurrent_dropout=0.25, return_sequences=False, kernel_initializer="random_uniform"))
    model.add(Dense(num_classes, activation="softmax", kernel_initializer="random_uniform"))
    
    
    #Define loss, optimizer, and metrics
    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    return model

In [6]:
def train(trainData, trainLabel, model):
    
    batch_size = 32  #the number of data to feed into model per batch
    num_epoch = 100  # go through your training data epoch times
    
    #callbacks stop training if val_loss is not improving.
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose = 1)]
    
    history = model.fit(trainData, trainLabel, validation_split=0.2, epochs = num_epoch, batch_size=batch_size, verbose=1, callbacks = callbacks)
                 
    return history, model

In [7]:
def evaluate(validData, validLabel, model):
    loss, accuracy = model.evaluate(validData, validLabel)
    return loss, accuracy

In [8]:
def predict(testData, model):
    result = model.predict_classes(testData, verbose=1)
    return result

DATA Preprocess

In [9]:
trainData = np.expand_dims(np.array(featureProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\train_features.csv")), axis=2)
validData = np.expand_dims(np.array(featureProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\valid_features.csv")), axis=2)
testData = np.expand_dims(np.array(featureProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\test_features.csv")), axis=2)

In [10]:
trainLabel = labelProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\train_labels.csv")
validLabel = labelProcess(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\valid_labels.csv")
#get classes
classes = list(set(trainLabel))

oneHotTrain = np.array(toOneHot(classes, trainLabel))
oneHotvalid = np.array(toOneHot(classes, validLabel))

Training

In [11]:
model = LSTMModel(trainData, oneHotTrain)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 148, 128)          1408      
_________________________________________________________________
dropout_1 (Dropout)          (None, 148, 128)          0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 74, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 74, 64)            81984     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                23000     
_________________________________________________________________
dense_1 (Dense)      

In [12]:
history = train(trainData, oneHotTrain, model)
print("Finish training")

Instructions for updating:
Use tf.cast instead.
Train on 6142 samples, validate on 1536 samples
Epoch 1/100
6142/6142 [==============================] - 33s 5ms/step - loss: 1.8674 - acc: 0.2867 - val_loss: 1.7535 - val_acc: 0.3372
Epoch 2/100
6142/6142 [==============================] - 29s 5ms/step - loss: 1.6942 - acc: 0.3730 - val_loss: 1.6358 - val_acc: 0.4082
Epoch 3/100
6142/6142 [==============================] - 31s 5ms/step - loss: 1.6550 - acc: 0.3818 - val_loss: 1.6013 - val_acc: 0.4219
Epoch 4/100
6142/6142 [==============================] - 30s 5ms/step - loss: 1.6400 - acc: 0.3865 - val_loss: 1.6011 - val_acc: 0.4173
Epoch 5/100
6142/6142 [==============================] - 30s 5ms/step - loss: 1.6302 - acc: 0.3951 - val_loss: 1.6012 - val_acc: 0.4258
Epoch 6/100
6142/6142 [==============================] - 31s 5ms/step - loss: 1.6083 - acc: 0.3992 - val_loss: 1.5807 - val_acc: 0.4290
Epoch 7/100
6142/6142 [==============================] - 32s 5ms/step - loss: 1.5923 - a

In [13]:
loss, accuracy = evaluate(validData, oneHotvalid, model)
print("loss, accuracy:", loss, accuracy)

450/450 [==============================] - 1s 2ms/step
loss, accuracy: 1.3891947831047906 0.4866666666666667


Predict

In [14]:
trackID = pd.read_csv(r"D:\Unimelb\Introduction to Machine Learning\assignment2\dataset\dataset\test_features.csv").loc[:, "trackID"].values.tolist()
prediction = predict(testData, model)

428/428 [==============================] - 1s 2ms/step


In [15]:
results_name = []

for result in prediction:
    results_name.append(classes[result])
result = list(zip(trackID,results_name))

In [17]:
with open("audio_CNN+LSTM_outPut.csv", 'w', newline='') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(["trackID", "genre"])
    writer.writerows(result)

train all feature
1. 41.78
2. 43.33
3. 44
4. 44.89
5. 42.22
6. 41.56
7. 42.45
8. 41.78
9. 43.78
10. 44.45

train MFCCs feature
1. 48.89
2. 45.34
3. 45.34
4. 44.89
5. 43.56
6. 44
7. 47.56
8. 45.11
9. 44.89
10. 43.11